# Model 1: Tipus incidència 'No ha impartit classe a aquest grup'
## Part-of-speech method

En este notebook se va ha intendar mejorar el modelo de detección de la incidencia 
'No ha impartit classe a aquest grup' mediante el uso de Part-of-Speech y Dependency Parsing.

Aquí se evaluarán las reglas definidas con los datos de muestra.

In [35]:
#import json
#import os.path
#import sys
import csv
import pandas as pd
#import numpy as np
#from colorama import Fore

In [2]:
#from __future__ import unicode_literals, print_function
#import plac
#import random
#from pathlib import Path

In [59]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher

In [25]:
def load_data(language):
    
    if (debug >= 1):
        print ("LOAD_DATA")
        
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print ("Original data:", data.shape[0])
    if (debug >= 2):
        display (data.sample(5))
        
    # Calculates label and filter rows to be used    
    data_prof = data[data.TipusPregunta == "P"][["Comentari","TipusIncidencia"]]
    if (debug >= 1):
        print("Filtered data: ", data_prof.shape[0])        
    if (debug >= 2):
        display (data_prof.sample(5))
        
    # Calculates labels
    data_prof["label"] = data_prof["TipusIncidencia"] == "No ha impartit classe a aquest grup"    
    data_fin = data_prof[["Comentari","label"]]
    
    if (debug >= 1):
        print ("Final data:", data_fin.shape[0])
    if (debug >= 2):
        display (data_fin.sample(5))
        
    return data_fin

In [61]:
def train_matcher(nlp):
    
    if (debug >= 1):
        print ("TRAIN MATCHER")
  
    # Rule-based matching
    matcher = Matcher(nlp.vocab, validate=True)

    # Defining patterns
    pattern = [{"TAG": "ADV__Polarity=Neg"}, {"OP": "*"}, 
           {"POS": "VERB", "LEMMA": {"IN": ["tratar","recibir","tener","dar","dictar","ir","conocer",
                                            "realizar","ver","saber","ser"]}}, 
           {"OP": "*"}, {"POS": "PRON","OP": "?"}]
    # Example: "No he tratado", "No traté", "No he recibido", "No recibí"
    matcher.add("NoClasses1", None, pattern)
    
    pattern = [{"TAG":"ADV__Polarity=Neg"}, {"OP": "*"}, {"POS":"NOUN", "LEMMA":{"IN":["clase", "asignatura"]}}]
    matcher.add("NoClasses2", None, pattern)
    
    if (debug >= 1):
        print ("Patterns: ", len(matcher))
    
    return matcher

In [70]:
def predict_text(nlp, matcher, text):
    
    if (debug >= 2):
        print ("PREDICT TEXT:")
    
    doc = nlp(text)
    matches = matcher(doc)
    
    if (len(matches) == 0):
        if (debug >= 2):
            print ("No matches")
        return False
    else:
        match_id, start, end = matches[len(matches)-1]
        string_id = nlp.vocab.strings[match_id] # Get string representation
        span = doc[start:end] # The matched span
        if (debug >= 2):
            print (match_id, string_id, start, end, span.text)
        return True
    

In [79]:
def evaluate(nlp, matcher, data):
    fp_list = list()
    fn_list = list()

    if (debug == 1):
        print ("EVALUATE")

    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives

    for index, row in data.iterrows():
        text = row["Comentari"]
        label = row["label"]

        result = predict_text (nlp, matcher, text)

        if (label == True):
            if (result == True):
                tp += 1.0
            else:
                fn += 1.0
                fn_list.append(text)
            if (debug >= 2):
                print ("fn: ", text)    
        else:
            if (result == True):
                fp += 1.0
                fp_list.append(text)
                if (debug >= 2):
                    print ("fp: ", text)              
            else:
                tn += 1.0

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)

    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score, 
            "false_positive": fp_list, "false_negative": fn_list}
    
    
    
#    print (text, label, result)

In [82]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 2

In [83]:
languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", "en":"en_core_web_sm" }
#languages = {"es":"es_core_news_lg"}

language = "es"
model = languages[language]

# Load the model form spacy
nlp = spacy.load(model)

data = load_data(language)
matcher = train_matcher(nlp)
scores = evaluate(nlp, matcher, data)

print("Precission (p): ", scores["textcat_p"])
print("Recall (r): ", scores["textcat_r"])
print("F-score (f): ", scores["textcat_f"])
print("False positives: ", len(scores["false_positive"]))
print("False negatives: ", len(scores["false_negative"]))


LOAD_DATA
Original data: 1761


,Curs,TipusPregunta,TipusIncidencia,Comentari,IdiomaSpacy,IdiomaPyCld2,Idioma
843,2019-20,A,NaN,Se ha de centrar más en la materia ha explicar...,es,es,es
1165,2019-20,A,NaN,"La asignatura es interesante, pero en los powe...",es,es,es
220,2018-19,P,NaN,Excesiva carga de trabajo. Desatención sobre l...,es,es,es
939,2019-20,P,NaN,"En mi opinión, no organiza bien el tiempo, no ...",es,es,es
107,2018-19,A,NaN,La asignatura en general ha contribuido a mi d...,es,es,es


Filtered data:  942


,Comentari,TipusIncidencia
239,El aspecto positivo es su personalidad y el in...,No ha impartit classe a aquest grup
373,Es el único profesor que lleva una planificaci...,NaN
264,El profesor Brufal ha mostrado desde el primer...,NaN
255,Una profesora muy accesible y que siempre ha r...,NaN
126,profesor de cada estudiante....su metodología ...,NaN


Final data: 942


...

6239189525466108865 NoClasses1 10 61 no hemos sabido gran cosa, a parte de algún correo con mínima información y presentaciones que nos tenemos que preparar por nuestra cuenta. Creo que sobran más comentarios, porque si no está dando clase y estamos siendo autodidactas pues tampoco hay mucho más de lo que opinar.
fp:  Después de decretarse la cuarentena y fin de enseñanza presencial no hemos sabido gran cosa, a parte de algún correo con mínima información y presentaciones que nos tenemos que preparar por nuestra cuenta. Creo que sobran más comentarios, porque si no está dando clase y estamos siendo autodidactas pues tampoco hay mucho más de lo que opinar. 
PREDICT TEXT:
6239189525466108865 NoClasses1 55 146 no va acorde ni con la situación excepcional que estamos viviendo ni con la dificultad de la materia,  ya que no veo nada que sea favorecedor hacia el alumno o la alumna: menos clases (por no decir ninguna), menos de un minuto por pregunta y mismos métodos de calificación y evaluaci

6239189525466108865 NoClasses1 10 280 no ha habido ningún tipo de relación profesorado-alumnos. No he recibido respuesta a los correos escritos con dudas. Además de que hemos estado sin ningún tipo de información sobre los exámenes hasta finales de abril, cuando ya llevábamos más de un mes y medio de confinamiento. Se ha establecido el examen parcial para el 13 de mayo, dos semanas antes del supuesto final, el cual esta sin confirmar su fecha todavía, porque de nuevo no responden a los mensajes. Por lo tanto resulta muy difícil organizarse el tiempo, ya no solo tenemos esta asignatura. Además las condiciones del examen son del todo injustas, desde mi punto de vista, ya que tendremos un examen de 50 preguntas en 40 minutos, se necesita un mínimo de tiempo para razonar y menos de un minuto por pregunta no es suficiente. Por último me parece que utilizar el hecho de que la asistencia presencial es de solo un 10% en circunstancias normales, para desechar la idea de hacer clases online es m

No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
No matches
PREDICT TEXT:
6239189525466108865 NoClasses1 31 51 no son preguntas de ellas, son cogidas de otro profesor. No nos merecemos una profesora tan mala.
fp:  Molt malament. Una de las peores profesoras que he visto en mi vida. Su primer año y con muy pocas ganas de trabajar. 0 autocrítica. Sus exámenes no son preguntas de ellas, son cogidas de otro profesor. No nos merecemos una profesora tan mala.
PREDICT TEXT:
16617680501526927059 NoClasses2 0 17 No puedo valorar a a este profesor puesto que no ha aparecido en practicamente toda la asignatura
fn:  No puedo valorar a a este profesor puesto que no ha aparecido en practicamente toda la asignatura.
PREDICT TEXT:
No matches
PREDICT TEXT:
6239189525466108865 NoClasses1 0 9 No se sabe nada de el durante la cuaren

6239189525466108865 NoClasses1 13 54 no sirve para nada. No nos han ayudado a aprender a programar, el temario solo sirve para poder hacer su trabajo que por cierto no tiene ningún sentido como está planteado. No muestra simpatía con los alumos.
fp:  Aspectos a mejorar. Debería actualizarse y cambiar el formato de enseñanza porque no sirve para nada. No nos han ayudado a aprender a programar, el temario solo sirve para poder hacer su trabajo que por cierto no tiene ningún sentido como está planteado. No muestra simpatía con los alumos. 
PREDICT TEXT:
No matches
PREDICT TEXT:
6239189525466108865 NoClasses1 8 110 no decir que es bastante nula. Apenas transmite conocimientos, y se limita a leer el power. Además no facilita para nada el aprendizaje de los alumnos con sus métodos de enseñanza. Por último, ni cumple sus horarios ni nos facilita la oportunidad de realizar tutorías con ella. Han sido más de una la clase que ha suspendido a una hora de comenzar esta, para después sin previo avi

6239189525466108865 NoClasses1 10 71 no poder realizar ninguna clase presencial debido a lo ocurrido, ha sabido sacrificarse y adaptarse para poder acercarse lo máximo posible a nosotros, dándonos los contenidos de una manera muy práctica y entretenida.
En estas clases y con la última conferencia que realizamos conjuntamente todos, he aprendido más que en muchas otras asignaturas de forma presencial
fp:  Quiero destacar el trabajo de Raul. A pesar de no poder realizar ninguna clase presencial debido a lo ocurrido, ha sabido sacrificarse y adaptarse para poder acercarse lo máximo posible a nosotros, dándonos los contenidos de una manera muy práctica y entretenida.
En estas clases y con la última conferencia que realizamos conjuntamente todos, he aprendido más que en muchas otras asignaturas de forma presencial
PREDICT TEXT:
6239189525466108865 NoClasses1 58 371 no hemos recibido un feedback individualizado. Para decirlo de forma más clara, y sin faltar a la verdad, el profesor no se ha 